In [8]:
#Importing nessary libraries
#Importing nessary libraries
import os
import datetime
import shutil
import sys
import pandas as pd
import time
import numpy as np
import calendar
from calendar import monthrange
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")
from importlib import reload
import pyodbc
import pandas as pd

reload(sys)
pd.options.mode.chained_assignment = None

start = time.time()

path = r"\\filesrvwhq\PowerWorks_Ops\Ambulatory Services\Client and Team Folders\0_New_Client_Folders\Benewah_BENE_ID\2020\11_November\11_14_2020\EATB Weekly - WE 11.14.20 - BENE_ID.xlsx"
df_m = pd.read_excel(path, sheet_name = "Enhanced_ATB")
df_m.rename(columns = {"Responsible Health Plan": "Current Health Plan", "Responsible Financial Class" : "Current Financial Class", "Balance Amount" : "Encounter Balance"}, inplace = True)
print("Datatype of 'Encounter Number' in ATB - ", df_m['Encounter Number'].dtype)
df = copy.deepcopy(df_m)
df['Encounter Number'].fillna('Blanks', inplace=True)
df['Encounter Number'] = df['Encounter Number'].astype(str)
df['Balance Type 1'] = None
df['Current Financial Class'].fillna((df['Primary Financial Class']), inplace=True)
df['Balance Type 1'][df['Current Financial Class'].isin(['Self Pay', 'SELF PAY', 'Self Pay / Unknown', 'Self Pay Other Coverage Pending']) == True] = 'Self Pay'
df['Balance Type 1'][df['Current Financial Class'].isin(['Self Pay', 'SELF PAY', 'Self Pay / Unknown', 'Self Pay Other Coverage Pending']) == False] = 'Insurance'
print('\nUpdated - Balance Type')
df['Category'] = None
df['Category'][(df['Insurance Balance'] <= 0) & (df['Balance Type 1'] == 'Insurance')] = 'Credit Balance - Insurance'
df['Category'][(df['Insurance Balance'] <= 0) & (df['Balance Type 1'] == 'Self Pay')] = 'Credit Balance - Self Pay'
df['Category'][(df['Discharge Aging Category'] == 'DNFB') & (df['Balance Type 1'] == 'Insurance') & df['Category'].isnull()] = 'DNFB - Insurance'
df['Category'][(df['Discharge Aging Category'] == 'DNFB') & (df['Balance Type 1'] == 'Self Pay') & df['Category'].isnull()] = 'DNFB - Self Pay'
df['Category'][(df['Discharge Aging Category'] == 'Not Aged') & (df['Balance Type 1'] == 'Insurance') & df['Category'].isnull()] = 'Not Aged - Insurance'
df['Category'][(df['Discharge Aging Category'] == 'Not Aged') & (df['Balance Type 1'] == 'Self Pay') & df['Category'].isnull()] = 'Not Aged - Self Pay'
df['Category'][(df['Balance Type 1'] == 'Self Pay') & df['Category'].isnull()] = 'Self Pay'
df['Category'][(df['Balance Type 1'] == 'Insurance') & df['Category'].isnull()] = 'Insurance'
print('\nUpdated - Category')
df['FBD Age'] = None
df['FBD Age'] = (pd.to_datetime((df['Activity Date']), format='%m/%d/%Y') - pd.to_datetime((df['First Claim Submission Date']), format='%m/%d/%Y')).dt.days
print('\nUpdated - First billed date age')
df['FBD Aging'] = None
df['FBD Aging'][df['FBD Age'].isin(list(range(0, 31)))] = '0-30'
df['FBD Aging'][df['FBD Age'].isin(list(range(31, 91)))] = '31-90'
df['FBD Aging'][df['FBD Age'].isin(list(range(91, 181)))] = '91-180'
df['FBD Aging'][df['FBD Age'].isin(list(range(181, 366)))] = '181-365'
df['FBD Aging'][df['FBD Age'].isin(list(range(366, 43784)))] = '366+'
print('\nUpdated - First billed date aging bucket')
df['LBD Age'] = None
df['LBD Age'] = (pd.to_datetime((df['Activity Date']), format='%m/%d/%Y') - pd.to_datetime((df['Last Claim Transmission Date']), format='%m/%d/%Y')).dt.days
print('\nUpdated - Last billed date age')
df['LBD Aging'] = None
df['LBD Aging'][df['LBD Age'].isin(list(range(0, 31)))] = '0-30'
df['LBD Aging'][df['LBD Age'].isin(list(range(31, 91)))] = '31-90'
df['LBD Aging'][df['LBD Age'].isin(list(range(91, 181)))] = '91-180'
df['LBD Aging'][df['LBD Age'].isin(list(range(181, 366)))] = '181-365'
df['LBD Aging'][df['LBD Age'].isin(list(range(366, 43784)))] = '366+'
print('\nUpdated - Last billed date aging bucket')
df['DOS Age'] = None
df['DOS Age'] = (pd.to_datetime((df['Activity Date']), format='%m/%d/%Y') - pd.to_datetime((df['Discharge Date']), format='%m/%d/%Y')).dt.days
df['DOS Age'][df['Discharge Date'].isnull()] = 0
df['DOS Aging Bucket'] = None
df['DOS Aging Bucket'][(df['DOS Age'] >= 0) & (df['DOS Age'] < 31)] = 'A. 0 to 30 days'
df['DOS Aging Bucket'][(df['DOS Age'] >= 31) & (df['DOS Age'] < 61)] = 'B. 31 to 60 days'
df['DOS Aging Bucket'][(df['DOS Age'] >= 61) & (df['DOS Age'] < 91)] = 'C. 61 to 90 days'
df['DOS Aging Bucket'][(df['DOS Age'] >= 91) & (df['DOS Age'] < 121)] = 'D. 91 to 120 days'
df['DOS Aging Bucket'][(df['DOS Age'] >= 121) & (df['DOS Age'] < 151)] = 'E. 121 to 150 days'
df['DOS Aging Bucket'][(df['DOS Age'] >= 151) & (df['DOS Age'] < 181)] = 'F. 151 to 180 days'
df['DOS Aging Bucket'][(df['DOS Age'] >= 181) & (df['DOS Age'] < 366)] = 'G. 181 to 365 days'
df['DOS Aging Bucket'][df['DOS Age'] >= 366] = 'H. 366 and above'
df['DOS Aging Bucket'].replace('', 'A. 0 to 30 days', inplace=True)
print('\nUpdated - DOS Aging Bucket')
df['Worked in last 15 days'] = None
df['Worked in last 15 days'] = (pd.to_datetime((df['Activity Date']), format='%m/%d/%Y') - pd.to_datetime((df['Encounter Last Touch Date']), format='%m/%d/%Y')).dt.days
df['Worked in last 15 days Bucket'] = None
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 0) & (df['Worked in last 15 days'] < 16)] = 'A. 0 to 15 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 16) & (df['Worked in last 15 days'] < 31)] = 'B. 15 to 30 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 31) & (df['Worked in last 15 days'] < 61)] = 'C. 31 to 60 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 61) & (df['Worked in last 15 days'] < 91)] = 'D. 61 to 90 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 91) & (df['Worked in last 15 days'] < 121)] = 'E. 91 to 120 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 121) & (df['Worked in last 15 days'] < 151)] = 'F. 121 to 150 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 151) & (df['Worked in last 15 days'] < 181)] = 'G. 151 to 180 days'
df['Worked in last 15 days Bucket'][(df['Worked in last 15 days'] >= 181) & (df['Worked in last 15 days'] < 366)] = 'H. 181 to 365 days'
df['Worked in last 15 days Bucket'][df['Worked in last 15 days'] >= 366] = 'F. 366 and above'
df['Worked in last 15 days Bucket'].replace('', 'A. 0 to 30 days', inplace=True)
print('\nUpdated - Worked in last 15 days Bucket')
df['Final Status'] = None
df['Final Status'][(df['Insurance Balance'] <= 0) & (df['Discharge Aging Category'].isin(['DNFB', 'Not Aged']) == False)] = 'Self Pay/Zero Balance'
df['Final Status'][df['Discharge Aging Category'].isin(['DNFB', 'Not Aged']) & df['Final Status'].isnull()] = 'Not-Factored'
df['Final Status'][(df['FBD Aging'] == '0-30') & df['Final Status'].isnull()] = 'FBD 0-30'
df['Final Status'][(df['LBD Aging'] == '0-30') & df['Final Status'].isnull()] = 'LBD 0-30'
df['Final Status'][(df['DOS Aging Bucket'] == 'A. 0 to 30 days') & df['Final Status'].isnull()] = 'Discharge 0-30'
df['Final Status'][df['Last Denial Reason'].notnull() & df['Final Status'].isnull()] = 'Technical Denial'
df['Final Status'][(df['Worked in last 15 days Bucket'] == 'A. 0 to 15 days') & df['Final Status'].isnull()] = 'Worked in last 15 days'
df['Final Status'][df['Final Status'].isnull()] = 'Workable'
print('\nUpdated - Final Status')
df['Status'] = None
df['Status'][df['Final Status'] == 'Self Pay/Zero Balance'] = 'Not-Factored'
df['Status'][df['Final Status'] == 'Not-Factored'] = 'Not-Factored'
df['Status'][df['Final Status'] == 'FBD 0-30'] = 'Non-Workable'
df['Status'][df['Final Status'] == 'LBD 0-30'] = 'Non-Workable'
df['Status'][df['Final Status'] == 'Discharge 0-30'] = 'Non-Workable'
df['Status'][df['Final Status'] == 'Worked in last 15 days'] = 'Non-Workable'
df['Status'][df['Final Status'] == 'Technical Denial'] = 'Workable'
df['Status'][df['Final Status'] == 'Workable'] = 'Workable'
print('\nUpdated - Status')
print('\nCreating the pivot table...')
df_t = pd.pivot_table(df, index=['Final Status', 'Status'], columns=['DOS Aging Bucket'], values=['Encounter Number'], aggfunc='count', fill_value=0)
print('\nCurrent ATB_Pivot has been created')
print('\nReport generated! Saving the file...')
dfn = pd.ExcelWriter(local_path + '\\' + i.split("- ")[-1])
df.to_excel(dfn, sheet_name='Worked report', encoding='cp1251', index=False)
df_t.to_excel(dfn, sheet_name='Piovt', encoding='cp1251')
dfn.save()
dfn.close()
print('\nWorked report has been saved.\nPlease wait...')
end = time.time()
time_taken = (end - start) / 60
print('\nTime taken - ', round(time_taken, 2), 'minutes')
print("{} is updated now and ready to use".format(i.split("- ")[-1]))

total_time = (time.time() - start)/60
print("All the productivity files are updated now")
print(f"Total time taken by program to run is {total_time} minutes")

Datatype of 'Encounter Number' in ATB -  object

Updated - Balance Type

Updated - Category

Updated - First billed date age

Updated - First billed date aging bucket

Updated - Last billed date age

Updated - Last billed date aging bucket

Updated - DOS Aging Bucket

Updated - Worked in last 15 days Bucket

Updated - Final Status

Updated - Status

Creating the pivot table...

Current ATB_Pivot has been created

Report generated! Saving the file...


NameError: name 'local_path' is not defined

In [107]:
(path.split(".")[-2]).split("- ")[-1]

'BENE_ID'

In [113]:
dff = df_t.copy()

In [114]:
dff.columns = list(map("_".join, dff.columns))

In [115]:
dff.index = list(map("_".join, dff.index))

In [116]:
dff['Final Status'] = dff.index

In [82]:
dff.to_excel(r'C:\Users\GN082282\OneDrive - Cerner Corporation\Desktop\ARC\boon.xlsx', index= False)

In [117]:
dff['Status'] = None

In [118]:
for i in range(0, len(dff["Final Status"])):
    dff['Status'][i] = dff['Final Status'][i].split("_")[0]
    

In [119]:
for i in range(0, len(dff["Final Status"])):
    dff['Final Status'][i] = dff['Final Status'][i].split("_")[1]

In [110]:
dff.to_numpy()

array([[129, 0, 0, 0, 0, 0, 0, 0, 'Non-Workable', 'Discharge 0-30',
        Timestamp('2020-11-14 00:00:00'), 'BENE_ID'],
       [1713, 237, 19, 17, 3, 1, 4, 4, 'Non-Workable', 'FBD 0-30',
        Timestamp('2020-11-14 00:00:00'), 'BENE_ID'],
       [1, 222, 58, 33, 23, 9, 26, 25, 'Non-Workable', 'LBD 0-30',
        Timestamp('2020-11-14 00:00:00'), 'BENE_ID'],
       [602, 104, 59, 21, 15, 3, 8, 1, 'Not-Factored', 'Not-Factored',
        Timestamp('2020-11-14 00:00:00'), 'BENE_ID'],
       [127, 420, 333, 305, 283, 192, 551, 364, 'Not-Factored',
        'Self Pay/Zero Balance', Timestamp('2020-11-14 00:00:00'),
        'BENE_ID'],
       [0, 104, 144, 99, 63, 53, 175, 113, 'Workable',
        'Technical Denial', Timestamp('2020-11-14 00:00:00'), 'BENE_ID'],
       [0, 227, 125, 44, 22, 15, 15, 35, 'Workable', 'Workable',
        Timestamp('2020-11-14 00:00:00'), 'BENE_ID'],
       [0, 75, 42, 17, 16, 10, 25, 26, 'Non-Workable',
        'Worked in last 15 days', Timestamp('2020-11-14 0

In [120]:
dff['Activity Date'] = df['Activity Date'][0]

In [121]:
dff['Client'] = (path.split(".")[-2]).split("- ")[-1]

In [123]:
dff = dff.to_numpy()

In [124]:
import pyodbc
#Connecting to SQL Server
server_name = "W1751904\LOCAL_CERNER"
database_name = "Test_Productivity_Dashboard"
conn = pyodbc.connect(Driver='{SQL Server Native Client 11.0}',
                      Server=server_name,
                      Database=database_name,
                      trusted_connection='yes')


cursor = conn.cursor()


#Uploading in SQL Action Code table
insert_query = """INSERT INTO Pivot_EATB ([0 to 30 days]
      ,[31 to 60 days]
      ,[61 to 90 days]
      ,[91 to 120 days]
      ,[121 to 150 days]
      ,[151 to 180 days]
      ,[181 to 365 days]
      ,[366 above]
      ,[Final Status]
      ,[Status],
       [Activity Date],
       [Client]) VALUES(?,?,?,?,?,?,?,?,?,?,?,?)
                    """


for row in dff:
    values = (row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11])
    cursor.execute(insert_query,values)

conn.commit()
print("EATB Pivot uploaded to SQL Server")


EATB Pivot uploaded to SQL Server


In [122]:
dff

,Encounter Number_A. 0 to 30 days,Encounter Number_B. 31 to 60 days,Encounter Number_C. 61 to 90 days,Encounter Number_D. 91 to 120 days,Encounter Number_E. 121 to 150 days,Encounter Number_F. 151 to 180 days,Encounter Number_G. 181 to 365 days,Encounter Number_H. 366 and above,Final Status,Status,Activity Date,Client
Discharge 0-30_Non-Workable,129,0,0,0,0,0,0,0,Non-Workable,Discharge 0-30,2020-11-14,BENE_ID
FBD 0-30_Non-Workable,1713,237,19,17,3,1,4,4,Non-Workable,FBD 0-30,2020-11-14,BENE_ID
LBD 0-30_Non-Workable,1,222,58,33,23,9,26,25,Non-Workable,LBD 0-30,2020-11-14,BENE_ID
Not-Factored_Not-Factored,602,104,59,21,15,3,8,1,Not-Factored,Not-Factored,2020-11-14,BENE_ID
Self Pay/Zero Balance_Not-Factored,127,420,333,305,283,192,551,364,Not-Factored,Self Pay/Zero Balance,2020-11-14,BENE_ID
Technical Denial_Workable,0,104,144,99,63,53,175,113,Workable,Technical Denial,2020-11-14,BENE_ID
Workable_Workable,0,227,125,44,22,15,15,35,Workable,Workable,2020-11-14,BENE_ID
Worked in last 15 days_Non-Workable,0,75,42,17,16,10,25,26,Non-Workable,Worked in last 15 days,2020-11-14,BENE_ID
